# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
!pip install folium


In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium
import json
import os
import numpy as np


# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../python-api-challenge/City_Data.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ust-nera,64.5667,143.2000,-19.69,87,59,0.47,RU,1742435582
1,1,grytviken,-54.2811,-36.5092,3.11,90,62,1.98,GS,1742435583
2,2,sittwe,20.1500,92.9000,25.27,78,54,1.43,MM,1742435585
3,3,blackmans bay,-43.0167,147.3167,28.74,32,100,0.89,AU,1742435586
4,4,nova sintra,14.8667,-24.7167,20.70,80,0,12.04,CV,1742435587


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
#%%capture --no-display

# Configure the map plot

# Create a map using state centroid coordinates to set the center of the map
city_map = folium.Map(location = [0, 0], zoom_start=2)

#Plot each city on the map
for index, row in city_data_df.iterrows():

    #Create a circle marker for each city.
    folium.CircleMarker(
        location = [row["Lat"], row["Lng"]],
        radius = row["Humidity"] / 12,

        #Configure popup
        popup = (
            f"City: {row['City']}<br>"
            f"Country: {row['Country']}<br>"
            f"Humidity: {row['Humidity']}%"
        ),
        color = "green",
        fill = True,
        fill_color = "skyblue",
        fill_opacity = 0.8
    ).add_to(city_map)

# Display the map
city_map

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = city_data_df[

    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 8) &
    (city_data_df["Cloudiness"] == 0)
]

ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,port saint john's,-31.6229,29.5448,21.78,87,0,3.93,ZA,1742435602
61,61,port elizabeth,-33.9180,25.5701,22.95,64,0,1.54,ZA,1742435655
67,67,upington,-28.4478,21.2561,22.21,68,0,1.54,ZA,1742435661
101,101,progreso,21.2833,-89.6667,24.41,80,0,6.58,MX,1742435700
106,106,port alfred,-33.5906,26.8910,23.41,60,0,2.97,ZA,1742435705


In [27]:
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,port saint john's,-31.6229,29.5448,21.78,87,0,3.93,ZA,1742435602
61,61,port elizabeth,-33.9180,25.5701,22.95,64,0,1.54,ZA,1742435655
67,67,upington,-28.4478,21.2561,22.21,68,0,1.54,ZA,1742435661
101,101,progreso,21.2833,-89.6667,24.41,80,0,6.58,MX,1742435700
106,106,port alfred,-33.5906,26.8910,23.41,60,0,2.97,ZA,1742435705


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

hotel_df.head()

,City,Country,Lat,Lng,Humidity
17,port saint john's,ZA,-31.6229,29.5448,87
61,port elizabeth,ZA,-33.9180,25.5701,64
67,upington,ZA,-28.4478,21.2561,68
101,progreso,MX,21.2833,-89.6667,80
106,port alfred,ZA,-33.5906,26.8910,60


In [30]:
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
17,port saint john's,ZA,-31.6229,29.5448,87,
61,port elizabeth,ZA,-33.9180,25.5701,64,
67,upington,ZA,-28.4478,21.2561,68,
101,progreso,MX,21.2833,-89.6667,80,
106,port alfred,ZA,-33.5906,26.8910,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params =  {
        "categories": "accommodation.hotel",
        "filter": f"circle:{lng},{lat},{radius}",
        "bias": f"proximity:{lng},{lat}",
        "limit": 1,
        "apiKey": geoapify_key
}
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port saint john's - nearest hotel: Outback Inn
port elizabeth - nearest hotel: Waterford Hotel
upington - nearest hotel: Die Eiland Holiday Ressort
progreso - nearest hotel: Centro Vacacional Obrero CTM
port alfred - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
general jose de san martin - nearest hotel: Hotel Belgrado
dickinson - nearest hotel: Days Inn by Wyndham Dickinson TX
cidreira - nearest hotel: Hotel Castelo
porto murtinho - nearest hotel: Hotel Saladero Cuê
ormara - nearest hotel: No hotel found
concepcion de la sierra - nearest hotel: No hotel found
gressier - nearest hotel: Venissia Bellissimo
saint-pierre - nearest hotel: Tropic Hotel
east london - nearest hotel: No hotel found
mahad - nearest hotel: Sagar Hotel
mahabaleshwar - nearest hotel: No hotel found
coxen hole - nearest hotel: Elizabeth Inn
lionel town - nearest hotel: No hotel found
tura - nearest hotel: Hotel Polo Orchid
nova vicosa - nearest hotel: Pousada Pontal da Barra

,City,Country,Lat,Lng,Humidity,Hotel Name
17,port saint john's,ZA,-31.6229,29.5448,87,Outback Inn
61,port elizabeth,ZA,-33.9180,25.5701,64,Waterford Hotel
67,upington,ZA,-28.4478,21.2561,68,Die Eiland Holiday Ressort
101,progreso,MX,21.2833,-89.6667,80,Centro Vacacional Obrero CTM
106,port alfred,ZA,-33.5906,26.8910,60,No hotel found
110,pisco,PE,-13.7000,-76.2167,73,La Portada
112,general jose de san martin,AR,-26.5374,-59.3416,37,Hotel Belgrado
147,dickinson,US,29.4608,-95.0513,29,Days Inn by Wyndham Dickinson TX
192,cidreira,BR,-30.1811,-50.2056,74,Hotel Castelo
199,porto murtinho,BR,-21.6989,-57.8825,63,Hotel Saladero Cuê


In [34]:
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port saint john's,ZA,-31.6229,29.5448,87,Outback Inn
1,port elizabeth,ZA,-33.9180,25.5701,64,Waterford Hotel
2,upington,ZA,-28.4478,21.2561,68,Die Eiland Holiday Ressort
3,progreso,MX,21.2833,-89.6667,80,Centro Vacacional Obrero CTM
4,port alfred,ZA,-33.5906,26.8910,60,No hotel found
5,pisco,PE,-13.7000,-76.2167,73,La Portada
6,general jose de san martin,AR,-26.5374,-59.3416,37,Hotel Belgrado
7,dickinson,US,29.4608,-95.0513,29,Days Inn by Wyndham Dickinson TX
8,cidreira,BR,-30.1811,-50.2056,74,Hotel Castelo
9,porto murtinho,BR,-21.6989,-57.8825,63,Hotel Saladero Cuê


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

# Configure the map plot

hotel_vacation_map = folium.Map(location=[0, 0], zoom_start=2)

for index, row in hotel_df.iterrows():
    hotel = row["Hotel Name"]
    city = row["City"]
    country = row["Country"]
    lat = row["Lat"]
    lon = row["Lng"]

    popup_text = (
        f"<b>Hotel:</b> {hotel}<br>"
        f"<b>City:</b> {city}<br>"
        f"<b>Country:</b> {country}"
    )

#Add marker to the map

    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color="blue", icon="bed")
    ).add_to(hotel_vacation_map)


# Display the map
hotel_vacation_map